# Pneuma: Quick Start

In this notebook, we show how to use each of Pneuma's features, from registering a dataset to querying the index.

## Offline Stage

In the offline stage, we set up Pneuma, including initializing the database, registering dataset and metadata, generating summaries, and generating both vector and keyword index.

To use pneuma, we import the class Pneuma from the pneuma module. 
- CUBLAS_WORKSPACE_CONFIG is set to a certain value (in this demo `:4096:8`) to enforce more deterministic behavior in cuBLAS operations.
- CUDA_VISIBLE_DEVICES is used to select the GPU. 
- The out_path is used to determine where the dataset and indexes will be stored. If not set, it will be defaulted to ~/.local/share/Pneuma/out on Linux or /Documents/Pneuma/out on Windows.

**NOTE**: In Colab, you will need to restart the runtime after installing the requirements.

In [ ]:
import json
import os

os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

from pneuma import Pneuma

We initialize the pneuma object with out_path and call the setup() function to initialize the database.

In [ ]:
out_path = "out_demo"
pneuma = Pneuma(
    out_path=out_path,
    llm_path="Qwen/Qwen2.5-7B-Instruct",
    embed_path="BAAI/bge-base-en-v1.5",
)
pneuma.setup()

### Registration

To register a dataset, we call the add_tables function while pointing to a directory and specifying the data creator.

In [ ]:
data_path = "data_src/sample_data/csv"
response = pneuma.add_tables(path=data_path, creator="demo_user")
json.loads(response)

Then, we can summarize the tables, all of which are not yet summarized at this point. These summaries then represent the tables for the discovery process.

In [ ]:
response = pneuma.summarize()
json.loads(response)

Optionally, if context (metadata) is available, we can register it as well using the add_metadata function.

In [ ]:
metadata_path = "data_src/sample_data/metadata.csv"
response = pneuma.add_metadata(metadata_path=metadata_path)
json.loads(response)

### Index Generation
To generate both vector and keyword index, we call the generate_index function while specifying a name for the index. By default, this function will index all registered tables.

In [ ]:
response = pneuma.generate_index(index_name="demo_index")
json.loads(response)

## Online Stage (Querying)
To retrieve a ranked list of tables, we use the query_index function.

In [ ]:
queries = [
    "Which dataset contains climate issues?",  # 5n77-2d6a.csv
    "If I could identify where the bus stops are in Chicago, that would be awesome!"  # 5cq6-qygt.csv
]

response = pneuma.query_index(
    index_name="demo_index",
    queries=queries,
    k=1,
    n=5,
    alpha=0.5,
)
relevant_tables = json.dumps(json.loads(response), indent=4)
print(relevant_tables)